# Better Way 35. 메타클래스로 클래스 속성에 주석을 달자

* 클래스를 정의한 이후 그 클래스를 실제로 사용하기 전 프로퍼티를 수정하거나 주석을 붙이는 것.
    - 메타클래스로 구현할 수 있는 기능 중 하나.
    - 보통 디스크립터(BW31 참조)와 함께 사용.
        + 클래스에서 디스크립터를 어떻게 사용하는지 자세히 조사한 정보를 디스크립터에 제공.

* 예: 고객 데이터베이스의 로우를 표현하는 새 클래스를 정의.
    - 데이터베이스 테이블의 각 칼럼에 대응하는 클래스의 프로퍼티가 존재해야 함.
    - 프로퍼티를 칼럼 이름과 연결하는 데 사용할 디스크립터 클래스를 정의

In [1]:
class Field(object):
    def __init__(self, name):
        self.name = name
        self.internal_name = '_' + self.name
    
    def __get__(self, instance, instance_type):
        if instance is None: return self
        return getattr(instance, self.internal_name, '')
    
    def __set__(self, instance, value):
        setattr(instance, self.internal_name, value)

* Field 디스크립터에 저장할 칼럼 이름이 있는 경우.
    - 좋지 않은 방법(필드 이름을 수동으로 여러 번 지정해야 함): 내장 함수 setattr, getattr을 사용해 모든 인스턴스별 상태를 인스턴스 딕셔너리에 보호 필드로 직접 저장 가능.
        + 처음에는 메모리 누수를 피하기 위해 weakref으로 디스크립터를 만드는 것보다 편리해 보임.


* 로우를 표현하는 클래스를 정의할 때는 각 클래스 속성에 대응하는 칼럼의 이름을 지정해야 함.

In [2]:
class Customer(object):
    # class property
    first_name = Field('first_name')
    last_name = Field('last_name')
    prefix = Field('prefix')
    suffix = Field('suffix')

In [4]:
foo = Customer()
print('Before:', repr(foo.first_name), foo.__dict__)
foo.first_name = 'Euclid'
print('After:', repr(foo.first_name), foo.__dict__)

Before: '' {}
After: 'Euclid' {'_first_name': 'Euclid'}


* 중복 문제
    - 이미 class 문 본문에서 Field 객체를 생성하여 Customer.first_name에 할당할 때 필드의 이름을 선언했음.
    - 그러나 필드 이름(‘first_name’)을 Field 생성자에도 넘겨야 함.

* 원인?
    - Customer 클래스 정의에서 연산 순서는 왼쪽에서 오른쪽으로 읽는 방식과 **반대**.
    - 먼저 Field 생성자는 Field(‘first_name’) 형태로 호출.
    - 다음으로 이 호출의 반환 값을 Customer.field_name에 할당.
    - 따라서 Field에서는 자신이 어던 클래스 속성에 할당될지 미리 알 방법이 없음.

* 해결책?
    - 메타클래스 사용!
        + class 문을 직접 후킹하여 class 본문이 끝나자마자 원하는 동작을 처리 가능.


* 예: 필드 이름을 수동으로 여러 번 지정하지 않고 메타클래스를 사용해 Field.name과 Field.internal_name을 디스크립터에 자동으로 할당하기.

In [5]:
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        for key, value in class_dict.items():
            if isinstance(value, Field):
                value.name = key
                value.internal_name = '_' + key
        cls = type.__new__(meta, name, bases, class_dict)
        return cls

* 메타클래스를 사용하는 기반 클래스를 정의한 코드.
    - 데이터베이스 로우를 표현하는 클래스가 모두 이 클래스를 상속하게 해 모두 메타클래스를 사용하게 해야 함.

In [6]:
class DatabaseRow(object, metaclass=Meta):
    pass

* 메타클래스를 사용하게 해도 필드 디스크립터는 거의 변경이 없음.
    - 유일한 차이?
        + 더는 생성자에 인수를 넘길 필요가 없음.
        + 필드 디스크립터의 속성은 Meta.\__new\__ 메서드로 설정됨.

In [7]:
class Field(object):
    def __init__(self):
        # metaclass assigns these attributes
        self.name = None
        self.internal_name = None
        
    def __get__(self, instance, instance_type):
        if instance is None: return self
        return getattr(instance, self.internal_name, '')

    def __set__(self, instance, value):
        setattr(instance, self.internal_name, value)

In [8]:
class BetterCustomer(DatabaseRow):
    first_name = Field()
    last_name = Field()
    prefix = Field()
    suffix = Field()

* 새로 작성한 클래스는 기존에 작성한 것과 동일하게 작동함.

In [9]:
foo = BetterCustomer()
print('Before:', repr(foo.first_name), foo.__dict__)
foo.first_name = 'Euler'
print('After: ', repr(foo.first_name), foo.__dict__)

Before: '' {}
After:  'Euler' {'_first_name': 'Euler'}


## 핵심 정리
* **메타클래스**를 이용하면 **클래스가 완전히 정의되기 전에 클래스 속성을 수정**할 수 있다.
* **디스크립터**와 **메타클래스**는 **선언적 동작**과 **런타임 내부 조사(introspection)** 용으로 강력한 조합을 이룬다.
* 메타클래스와 디스크립터를 연계하여 사용하면 메모리 누수와 weakref 모듈을 모두 피할 수 있다.